In [1]:
# Start H2O-3 runtime.

import h2o
h2o.init(strict_version_check=False)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment Homebrew (build 11.0.21+0); OpenJDK 64-Bit Server VM Homebrew (build 11.0.21+0, mixed mode)
  Starting server from /Users/marek/git2/h2o-3/build/h2o.jar
  Ice root: /var/folders/bz/gzkngzwj2593j90gdmscv89c0000gn/T/tmpqhbfsn1i
  JVM stdout: /var/folders/bz/gzkngzwj2593j90gdmscv89c0000gn/T/tmpqhbfsn1i/h2o_marek_started_from_python.out
  JVM stderr: /var/folders/bz/gzkngzwj2593j90gdmscv89c0000gn/T/tmpqhbfsn1i/h2o_marek_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.99999
H2O_cluster_version_age:,17 hours and 36 minutes
H2O_cluster_name:,H2O_from_python_marek_u1r8vv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [2]:
# Configure DRF algorithm and train a model.

from h2o.estimators import H2ORandomForestEstimator

# Import the cars dataset into H2O:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")

# Set the predictors and response;
# set the response as a factor:
cars["economy_20mpg"] = cars["economy_20mpg"].asfactor()
predictors = ["displacement","power","weight","acceleration","year"]
response = "economy_20mpg"

# Split the dataset into a train and valid set:
train, valid = cars.split_frame(ratios=[.8], seed=1234)
drf = H2ORandomForestEstimator(ntrees=10,
                                    max_depth=5,
                                    min_rows=10,
                                    calibrate_model=True,
                                    calibration_frame=valid,
                                    binomial_double_trees=True)
drf.train(x=predictors,
          y=response,
          training_frame=train,
          validation_frame=valid)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1699962414622_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    10                 20                          3304                   4            5            4.45          7             10            8.55

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.05640340025564431
RMSE: 0.23749400046242075
LogLoss: 0.2919276910550563
Mean Per-Class Error: 0.07387110016420362
AUC: 0.971572249589491
AUCPR: 0.977918414938428
Gini: 0.9431444991789819

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5375000078231094
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      107  13   0.1083   (13.0/120.0)
1      8    195  0.0394   (8.0/203.0)
Total  115  208  0.065    (21.0/323.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.5375       0.948905  104
max f2                       0.267544     0.963567  127
max f0point5                 0.669643     0.95117   91
max accuracy                 0.608799     0.934985  98
max precision                0.975232     0.990385  8
max recall                   0.0195865    1         166
max specificity              1            0.991667  0
max absolute_mcc             0.608799     0.861032  98
max min_per_class_accuracy   0.669643     0.921182  91
max mean_per_class_accuracy  0.608799     0.93124   98
max tns                      1            119       0
max fns                      1            111       0
max fps                      0            120       168
max tps                      0.0195865    203       166
max tnr                      1            0.991667  0
max fnr                      1            0.546798  0
max fpr                      0            1         168
max tpr                      0.0195865    1         166

Gains/Lift Table: Avg response rate: 62.77 %, avg score: 63.00 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.286154                    1                  1.57601    1.57601            0.989247         1           0.989247                    1                   0.45098         0.45098                    57.6007   57.6007            0.442716
2        0.310769                    0.98               1.59314    1.57736            1                0.984959    0.990099                    0.998809            0.0392157       0.490196                   59.3137   57.7364            0.481932
3        0.406154                    0.936428           1.54175    1.569              0.967742         0.960521    0.984848                    0.989817            0.147059        0.637255                   54.1746   56.8999            0.620726
4        0.507692                    0.832413           1.44831    1.54486            0.909091         0.893657    0.969697                    0.970585            0.147059        0.784314                   44.8307   54.486             0.742991
5        0.609231                    0.639581           1.35175    1.51268            0.848485         0.767588    0.949495                    0.936752            0.13

In [3]:
# Log the model to an MLFlow reqistry.

import mlflow
import h2o_mlflow_flavor
mlflow.set_tracking_uri("http://127.0.0.1:8080")

with mlflow.start_run(run_name="cars") as run:
    mlflow.log_params(h2o_mlflow_flavor.get_params(drf)) # Log training parameters of the model (optional).
    mlflow.log_metrics(h2o_mlflow_flavor.get_metrics(drf)) # Log performance matrics of the model (optional).
    input_example = h2o_mlflow_flavor.get_input_example(drf) # Extract input example from training dataset (optional)
    h2o_mlflow_flavor.log_model(drf, "cars", input_example=input_example,
                                model_type="MOJO", # Specify whether the output model should be MOJO or POJO. (MOJO is default)
                                extra_prediction_args=["--predictCalibrated"]) # Add extra prediction args if needed.

/Users/marek/opt/anaconda3/envs/mlflow/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [4]:
# Load model from the MLFlow registry and score with the model.

import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:8080")

logged_model = 'runs:/a9ff364f07fa499eb44e7c49e47fab11/cars' # Specify correct id of your run.

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
data = pd.read_csv("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")
loaded_model.predict(data)

,predict,p0,p1,cal_p0,cal_p1
0,0,0.990000,0.010000,0.993618,0.006382
1,0,1.000000,0.000000,0.994100,0.005900
2,0,0.971818,0.028182,0.992639,0.007361
3,0,0.513932,0.486068,0.783478,0.216522
4,0,0.641056,0.358944,0.908093,0.091907
...,...,...,...,...,...
401,0,0.340244,0.659756,0.478411,0.521589
402,0,0.231999,0.768001,0.280546,0.719454
403,0,0.456524,0.543476,0.696867,0.303133
404,0,0.417457,0.582543,0.628017,0.371983
